In [28]:
import os
import requests
from dotenv import load_dotenv
import csv

load_dotenv()

tenant = 'miratech'
sector = "test"
# 'test' = UAT
# 'prod01' = PROD
# 'test02 = DEV
blueprint_name = "productConfigurator40"
rule_name = "kBFieldsObjectGenerator"

api_key = os.getenv("LOGIK_DEV_IO_API_KEY")

if not api_key:
    raise ValueError("API key not found. Please check your environment variables.")

In [29]:
def build_url(sector, endpoint):
    base_url = f"https://{tenant}.{sector}.logik.io"
    return f"{base_url}{endpoint}"

def make_api_request(url, api_key):
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

all_rules = []
page = 0
page_size = 100

while True:
    endpoint = f"/api/admin/v2/blueprints/{blueprint_name}/rules?page={page}&size={page_size}&sort=modified%2CDESC"
    url = build_url(sector, endpoint)
    rules_list = make_api_request(url, api_key)
    if rules_list and "content" in rules_list and rules_list["content"]:
        all_rules.extend(rules_list["content"])
        page += 1
    else:
        break

In [ ]:
if all_rules:
    csv_filename = f"{blueprint_name}_rules_export.csv"
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Name", "Variable Name", "Status", "Action Summary"])
        writer.writeheader()
        for rule in all_rules:
            if rule.get("status") == "active" and ("standardSelectionRunningFilters" in rule.get("variableName", "")
                                                   or "product40CalculateCostAdder" in rule.get("variableName", "")
                                                   or "product40RequiredFieldsMissing" in rule.get("variableName", "")
                                                   or "standardSelectionConfigProductReqFieldMsg" in rule.get("variableName", "")
                                                   or "product40ACCHide" in rule.get("variableName", "")):
                writer.writerow({
                    "Name": rule.get("name"),
                    "Variable Name": rule.get("variableName"),
                    "Status": rule.get("status"),
                    "Action Summary": rule.get("actionSummary")
                })
    print(f"Exported filtered rules to {csv_filename}")
else:
    print("No rules found.")


Exported filtered rules to productConfigurator40_rules_export.csv
